In [106]:
import os
import cx_Oracle
import pandas as pd
from pandas import DataFrame as df


ora_id = os.environ['ORACLE_ID']
ora_pw = os.environ['ORACLE_PW']
ora_ip = os.environ['ORACLE_IP']
ora_port = os.environ['ORACLE_PORT']
ora_sid = os.environ['ORACLE_SID']

# (예시) scott/tiger@127.0.0.1:1521/orcl
ora_addr = ora_id+'/' + ora_pw + '@' + ora_ip + ':' + ora_port + '/' + ora_sid

conn = cx_Oracle.connect(ora_addr)

cur_sel = conn.cursor()
cur_ins = conn.cursor()
cur_del = conn.cursor()


In [112]:
sql = 'SELECT * FROM HR.JOBS '

cur_sel.arraysize = 3

# 커서에 sql 셋팅
cur_sel.execute(sql)

# fetchsize 만큼 리스트 형태로 리턴 (정확히는 리스트안에 튜플들이 담겨있음.)
rows = cur_sel.fetchmany()
# rows = cur_sel.fetchmany(numRows=1)

# 더이상 결과가 없으면 빈리스트로 리턴.
while len(rows) != 0 :
    
    print(rows)
    
    # 여러 결과를 한번에 insert 처리.
    cur_ins.executemany("""
    INSERT INTO HR.JOBS2
    (JOB_ID, JOB_TITLE, MIN_SALARY, MAX_SALARY)
    VALUES(:1, :2, :3, :4)
    """, rows)

    conn.commit()
    
    rows = cur_sel.fetchmany()
    


[('AD_PRES', 'President', 20080, 40000), ('AD_VP', 'Administration Vice President', 15000, 30000), ('AD_ASST', 'Administration Assistant', 3000, 6000)]
[('FI_MGR', 'Finance Manager', 8200, 16000), ('FI_ACCOUNT', 'Accountant', 4200, 9000), ('AC_MGR', 'Accounting Manager', 8200, 16000)]
[('AC_ACCOUNT', 'Public Accountant', 4200, 9000), ('SA_MAN', 'Sales Manager', 10000, 20080), ('SA_REP', 'Sales Representative', 6000, 12008)]
[('PU_MAN', 'Purchasing Manager', 8000, 15000), ('PU_CLERK', 'Purchasing Clerk', 2500, 5500), ('ST_MAN', 'Stock Manager', 5500, 8500)]
[('ST_CLERK', 'Stock Clerk', 2008, 5000), ('SH_CLERK', 'Shipping Clerk', 2500, 5500), ('IT_PROG', 'Programmer', 4000, 10000)]
[('MK_MAN', 'Marketing Manager', 9000, 15000), ('MK_REP', 'Marketing Representative', 4000, 9000), ('HR_REP', 'Human Resources Representative', 4000, 9000)]
[('PR_REP', 'Public Relations Representative', 4500, 10500)]


In [117]:
sql = 'SELECT * FROM HR.JOBS2 '

df = pd.read_sql(sql, conn)
df

,JOB_ID,JOB_TITLE,MIN_SALARY,MAX_SALARY


In [115]:

sql = 'SELECT * FROM HR.JOBS '

cur_sel.arraysize = 3

# 커서에 sql 셋팅
cur_sel.execute(sql)

# fetchsize 만큼 리스트 형태로 리턴 (정확히는 리스트안에 튜플들이 담겨있음.)
row = cur_sel.fetchone()

while row is not None :
    print(row)
    # print(row[1])
    row = cur_sel.fetchone()
    
    



('AD_PRES', 'President', 20080, 40000)
('AD_VP', 'Administration Vice President', 15000, 30000)
('AD_ASST', 'Administration Assistant', 3000, 6000)
('FI_MGR', 'Finance Manager', 8200, 16000)
('FI_ACCOUNT', 'Accountant', 4200, 9000)
('AC_MGR', 'Accounting Manager', 8200, 16000)
('AC_ACCOUNT', 'Public Accountant', 4200, 9000)
('SA_MAN', 'Sales Manager', 10000, 20080)
('SA_REP', 'Sales Representative', 6000, 12008)
('PU_MAN', 'Purchasing Manager', 8000, 15000)
('PU_CLERK', 'Purchasing Clerk', 2500, 5500)
('ST_MAN', 'Stock Manager', 5500, 8500)
('ST_CLERK', 'Stock Clerk', 2008, 5000)
('SH_CLERK', 'Shipping Clerk', 2500, 5500)
('IT_PROG', 'Programmer', 4000, 10000)
('MK_MAN', 'Marketing Manager', 9000, 15000)
('MK_REP', 'Marketing Representative', 4000, 9000)
('HR_REP', 'Human Resources Representative', 4000, 9000)
('PR_REP', 'Public Relations Representative', 4500, 10500)


In [116]:

# delete
cur_del.execute('DELETE FROM HR.JOBS2')
conn.commit()

In [118]:
cur_sel.close()
cur_ins.close()
cur_del.close()

conn.close()